In [1]:
import os
import os.path as op
import json
import cv2
import base64
import numpy as np
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

from maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer
from maskrcnn_benchmark.structures.tsv_file_ops import generate_linelist_file
from maskrcnn_benchmark.structures.tsv_file_ops import generate_hw_file
from maskrcnn_benchmark.structures.tsv_file import TSVFile
from maskrcnn_benchmark.data.datasets.utils.image_ops import img_from_base64

In [3]:
orig_root = '../../../../datasets/coco_2014'
img_root = op.join(orig_root, 'images')

exp_root = '../../../../datasets/coco_my_mini_nn'

splits = ['train', 'val', 'test']

In [5]:
print(karpathy['images'][555]['filepath'])
print(karpathy['images'][555]['filename'])
print(karpathy['images'][555]['cocoid'])

KeyError: 'filepath'

In [5]:
karpathy_dict = {str(k['cocoid']): op.join(k['filepath'], k['filename']) for k in karpathy['images']}

karpathy_dict['382088']

'val2014/COCO_val2014_000000382088.jpg'

In [6]:
len(karpathy['images']), len(karpathy_dict)  # must be equal

(123287, 123287)

In [8]:
cap_source = '../../../../datasets/coco/coco_oscar_preexacted_vvl'
captions = {}

for split in splits:
    with open(op.join(cap_source, f'{split}_caption.json')) as f:
        captions[split] = json.load(f)

In [9]:
print(captions['test'][0])

{'image_id': '179765', 'id': 38, 'caption': 'A black Honda motorcycle parked in front of a garage.'}


In [10]:
img_list = {split: {} for split in splits}

for split in splits:
    for sample in tqdm(captions[split]):
        img_key = sample['image_id']
        img_fullpath = karpathy_dict[img_key]
        if img_key not in img_list[split]:
            img_list[split][img_key] = img_fullpath
        else:
            if img_list[split][img_key] != img_fullpath:
                print('Error!')
                
print(list(img_list['train'])[0], captions['train'][0])  # sample
print([len(img_list[split]) for split in splits])  # train, val, test

100%|██████████| 25010/25010 [00:00<00:00, 2010301.51it/s]

318556 {'image_id': '318556', 'id': 48, 'caption': 'A very clean and well decorated empty bathroom'}
[113287, 5000, 5000]


In [39]:
for split in splits[::-1]:
    rows = []
    rows_label = []
    rows_hw = []

    i = 50
    for sample in tqdm(img_list[split].items()):
        # print(sample)
        img_key = sample[0]
        img_path = op.join(img_root, sample[1])
        img = cv2.imread(img_path)
        img_encoded_str = base64.b64encode(cv2.imencode('.jpg', img)[1])
        row = (img_key, img_encoded_str)
        # print(row[1][800:900] , flush=True)
        rows.append(row)

        height = img.shape[0]
        width = img.shape[1]
        row_hw = [img_key, json.dumps([{'width': width, 'height': height}])]
        rows_hw.append(row_hw)
        i -= 1
        if i == 0:
            break
        
    exp_encoded_img_file = op.join(exp_root, f'{split}.img.tsv')
    exp_hw_file = op.join(exp_root, f'{split}.hw.tsv')
    tsv_writer(rows, exp_encoded_img_file)
    tsv_writer(rows_hw, exp_hw_file)
    # tsv_writer(rows_label, label_file)

  0%|          | 49/113287 [00:00<34:19, 54.98it/s]
